* Note: While this documentation uses the primitives from Qiskit Runtime, which allow you to use IBM backends, the primitives can be run on any provider by using the **backend primitives** instead. Additionally, you can use the *reference primitives* to run on a local statevector simulator  

Ths steps in this topic describe how to set up primitives, exploring the options you can use to configure them. and invoke them in a program  

* Using Fracional Gates: to use the newly supported **fractional gates**, set ```use_fractional_gates=True``` when requesting a backend from a ```QiskitRuntimeService``` instance. For example
```
service = QiskitRuntimeService()
fractional_gate_backend = service.least_busy(use_fractional_gates=True)
```

# Get started with Estimator

## 1. Initialize the account  
Since Qiskit Runtime Estimator is a managed service, you first need to initialize your account

In [5]:
from qiskit_ibm_runtime import QiskitRuntimeService

service =  QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator = False, min_num_qubits =127)

print(backend.name)

ibm_torino


## 2. Create a circuit and an observable

In [6]:
from qiskit.circuit.library import qaoa_ansatz
from qiskit.quantum_info import SparsePauliOp

entanglement = [tuple(edge) for edge in backend.coupling_map.get_edges()]

observable = SparsePauliOp.from_sparse_list(
    [('ZZ', [i,j], 0.5) for i, j in entanglement],
    num_qubits = backend.num_qubits)

In [7]:
circuit = qaoa_ansatz(observable, reps = 2)
param_values = [0.1, 0.2, 0.3, 0.4]
print(f">>> Observable: {observable.paulis}")

>>> Observable: ['IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...',
 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...',
 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...',
 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...',
 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...',
 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...',
 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...',
 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...',
 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...',
 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...',
 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...',
 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...',
 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...',
 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...',
 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...', ...]


The circuit and obsrevable need to be transformed to only use instructions supported by the QPU

In [13]:
from qiskit.transpiler import generate_preset_pass_manager
 
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
isa_circuit = pm.run(circuit)
isa_observable = observable.apply_layout(isa_circuit.layout)
print(f">>> Circuit ops (ISA): {isa_circuit.count_ops()}")

>>> Circuit ops (ISA): OrderedDict([('rz', 3826), ('sx', 1601), ('cz', 968)])


## 3. Initialzie Qiskit Runtime Estimator  
When you initialize the Estimator, use the ```mode``` parameter to specify the mode you want it to run it. Possible values are ```batch```, ```session```, or ```backend``` objects for batch, session, and job execution mode.

In [16]:
from qiskit_ibm_runtime import EstimatorV2 as Estimator
estimator = Estimator(mode = backend)

## 4. Invoe the Estimator and get results  
invoke the ```run()``` method to calculate expectation values for the input circuits and observables. The circuit, observable, and optional parameter value sets are input as *primitive unified bloc* (PUB) tuples

In [17]:
param_values

[0.1, 0.2, 0.3, 0.4]

In [18]:
job =estimator.run([(isa_circuit, isa_observable, param_values)])
print(f">>> Job ID: {job.job_id()}")
print(f">>> Job Status: {job.status()}")

>>> Job ID: d4b7a5hlag1s73bkut1g
>>> Job Status: QUEUED


In [27]:
result[0].data.evs

array(24.60678839)

In [19]:
result = job.result()
print(f">>> {result}")
print(f"  > Expectation value: {result[0].data.evs}")
print(f"  > Metadata: {result[0].metadata}")

>>> PrimitiveResult([PubResult(data=DataBin(evs=np.ndarray(<shape=(), dtype=float64>), stds=np.ndarray(<shape=(), dtype=float64>), ensemble_standard_error=np.ndarray(<shape=(), dtype=float64>)), metadata={'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32})], metadata={'dynamical_decoupling': {'enable': False, 'sequence_type': 'XX', 'extra_slack_distribution': 'middle', 'scheduling_method': 'alap'}, 'twirling': {'enable_gates': False, 'enable_measure': True, 'num_randomizations': 'auto', 'shots_per_randomization': 'auto', 'interleave_randomizations': True, 'strategy': 'active-accum'}, 'resilience': {'measure_mitigation': True, 'zne_mitigation': False, 'pec_mitigation': False}, 'version': 2})
  > Expectation value: 24.60678838751569
  > Metadata: {'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}


# Get started with Sampler

## 1. Initialize the account

In [28]:
from qiskit_ibm_runtime import QiskitRuntimeService
 
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)

## 2. Create a circuit

In [45]:
import numpy as np
from qiskit.circuit.library import efficient_su2
 
circuit = efficient_su2(num_qubits=127, reps=3, entanglement="linear") # 이 코드는 (ry rz + cnot) x (reps) + (ry rz) 구조
circuit.measure_all()
# The circuit is parametrized, so we will define the parameter values for execution
param_values = np.random.rand(circuit.num_parameters)

In [47]:
from qiskit.transpiler import generate_preset_pass_manager
 
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
isa_circuit = pm.run(circuit)
print(f">>> Circuit ops (ISA): {isa_circuit.count_ops()}")

>>> Circuit ops (ISA): OrderedDict([('sx', 3413), ('rz', 3036), ('cz', 1254), ('measure', 127), ('barrier', 1)])


## 3. Initialize the Qiskit Runtime Sampler

In [48]:
from qiskit_ibm_runtime import SamplerV2 as Sampler
sampler = Sampler(mode = backend)

## 4. Invoke the Sampler and get results

In [49]:
job = sampler.run([(isa_circuit, param_values)])
print(f">>> Job ID: {job.job_id()}")
print(f">>> Job Status: {job.status()}")

>>> Job ID: d4b7kt1lag1s73bkv780
>>> Job Status: QUEUED


In [ ]:
result = job.result()
 
# Get results for the first (and only) PUB
pub_result = result[0]
print(
    f"First ten results for the 'meas' output register: {pub_result.data.meas.get_bitstrings()[:10]}"
)

First ten results for the 'meas' output register: ['0001100010101110000011110111111000010110010001100110010110011100010100010010000010101101001010001100111001010010001000100000010', '1000000010001110001111101111001111000100000010000010010011011000001001000110001010010101100101010001000000001010001000000000010', '1100111110011101000010000001010000010111010111010110000111001111011000001010010100010100010001000001010010110000001100100000100', '1000110010110011111000000001111110100101110010000111000011000000001101011111011100000100000000010000000000000000000000000000000', '1000001100110100001010010011101100011001000010000111011101000110010100100000100100110101011100011101000010110111110010001100010', '1010101010000110010110111111110000101110001100000100000100000000000101010010010000100100110001000101000000010000001000001011000', '0001011000010001010110110010110010000001100000110110011110110110000000101000000010010000001111001010000001001101000101010001000', '0111001100000000110100110000110

# Get started with the backend primitives  
Unlike provider-specific primitives, backend primitives are generic implementations that can be used with an arbitrary ```backend``` object, as long as it implements the ```Backend``` interface.  
* The Sampler primitive can be run with any provider by using ```qiskit.primitives.BackendSamplerV2```
* The Estimator primitive can be run with any provider by using ```qiskit.primitives.BackendEstimatorV2``` 

In [ ]:
from qiskit.primitives import BackendEstimatorV2
from qiskit_aer import AerSimulator # == from <some_qiskit_provider> import QiskitProvider

backend = AerSimulator()
estimator = BackendEstimatorV2(backend = backend)


# from qiskit.primitives import BackendSamplerV2
# from <some_qiskit_provider> import QiskitProvider
 
# provider = QiskitProvider()
# backend = provider.get_backend('backend_name')
# sampler = BackendSamplerV2(backend)